# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [27]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [30]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [31]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [32]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor': 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [33]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [34]:
df = pd.read_csv('event_datafile_new.csv')
df

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38
5,Carrie Underwood,Gianna,F,2,Jones,195.70893,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Look At Me,38
6,No Te Va Gustar,Kaylee,F,0,Summers,246.43873,free,"Phoenix-Mesa-Scottsdale, AZ",833,Ya EntendÃÂ­,8
7,The Velvet Underground / Nico,Kaylee,F,1,Summers,360.09751,free,"Phoenix-Mesa-Scottsdale, AZ",833,All Tomorrow's Parties,8
8,Snoop Dogg / Jamie Foxx,Kaylee,F,2,Summers,179.35628,free,"Phoenix-Mesa-Scottsdale, AZ",833,Psst!,8
9,Dixie Chicks,Kaylee,F,3,Summers,201.01179,free,"Phoenix-Mesa-Scottsdale, AZ",833,There's Your Trouble,8


## Query 1

In [35]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT artist_name, song_title, song_length FROM events WHERE sessionId=338 AND itemInSession=4"

In [36]:
# Partitioning by sessionID as it contains multiple itemInSession's
create_table1 = """CREATE TABLE IF NOT EXISTS events 
                    (artist_name varchar, song_title text, song_length float, sessionId int, 
                    itemInSession int, 
                    PRIMARY KEY (sessionID, itemInSession))"""

In [37]:
session.execute(create_table1)

In [38]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO events "
        query += "(artist_name, song_title, song_length, sessionId, itemInSession) "
        query += "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [60]:
## Verify the data was entered into the table
rows= session.execute(query1)
pd.DataFrame([r for r in rows], columns=['artist_name', 'song_title', 'song_length'])

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2

In [62]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and 
# user (first and last name) for userid = 10, sessionid = 182
query2 = """SELECT artist_name, song_title, first_name, last_name
            FROM artists WHERE userId=10 AND sessionId=182"""                    

In [63]:
# Partitioning by userId because usersId's have sessionId's which have itemInSession's
create_table2 = """CREATE TABLE IF NOT EXISTS artists 
                   (artist_name varchar, song_title varchar, first_name text, last_name text, 
                    userID int, sessionId int, itemInSession int, 
                   PRIMARY KEY (userId, sessionId, itemInSession))"""

In [64]:
session.execute(create_table2)

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artists "
        query += "(artist_name, song_title, first_name, last_name, userID, sessionId, itemInSession) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [71]:
## Verify the data was entered into the table
rows= session.execute(query2)
pd.DataFrame([r for r in rows], columns=['artist_name', 'song_title', 'first_name', 'last_name'])

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3

In [151]:
## Query 3: Give me every user name (first and last) in my music app history who listened 
## to the song 'All Hands Against His Own'
query3 = """SELECT first_name, last_name
            FROM users
            WHERE song_title = 'All Hands Against His Own'"""                    

In [152]:
# Paritioning by song_title because that is what we're filtering by
# using userId as a composite key as it is more unique than using first_name and last_name
create_table3 = """CREATE TABLE IF NOT EXISTS users
                   (first_name text, last_name text, song_title varchar, userId int, 
                   PRIMARY KEY (song_title, userId))"""

In [153]:
session.execute(create_table3)

### Method 1: using csv.reader

In [85]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users "
        query += "(first_name, last_name, song_title, userID) VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [161]:
## Verify the data was entered into the table
rows = session.execute(query3)
pd.DataFrame([r for r in rows], columns=['first_name', 'last_name'])

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [162]:
# Double-check results
df[df['song']=='All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Method 2: Using pandas apply()

In [158]:
# Drop and recreate table
session.execute("DROP TABLE IF EXISTS users")
session.execute(create_table3)

# Create new dataframe
_df = df[['firstName', 'lastName', 'song', 'userId']].copy()
query = "INSERT INTO users "
query += "(first_name, last_name, song_title, userID) VALUES (%s, %s, %s, %s)"

# Use apply to iterate over the dataframe and pass values to cassandra
_df.apply(lambda x: session.execute(query, list(x.values)), axis=1)

## Verify the data was entered into the table
rows = session.execute(query3)
pd.DataFrame([r for r in rows], columns=['first_name', 'last_name'])

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Method 3: Using pandas iterrows()

In [159]:
# Drop and recreate table
session.execute("DROP TABLE IF EXISTS users")
session.execute(create_table3)

# Create new dataframe
_df = df[['firstName', 'lastName', 'song', 'userId']].copy()
query = "INSERT INTO users "
query += "(first_name, last_name, song_title, userID) VALUES (%s, %s, %s, %s)"

# Use iterrows to iterate over the dataframe and pass values to cassandra
for i, r in _df.iterrows():
    session.execute(query, list(r))

## Verify the data was entered into the table
rows = session.execute(query3)
pd.DataFrame([r for r in rows], columns=['first_name', 'last_name'])

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### TimeIt Comparisons

Method 1: using csv.reader

In [154]:
%%timeit
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users "
        query += "(first_name, last_name, song_title, userID) VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

6.03 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Method 2: Using pandas apply

In [149]:
%%timeit
_ = df3.apply(lambda x: session.execute(query, list(x.values)), axis=1)

6.24 s ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Method 3: using pandas iterrows()

In [144]:
%%timeit 
for i, r in df3.iterrows():
    session.execute(query, list(r))

8.26 s ± 59.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Conclusion: Using csv.reader is a little faster than pandas apply()

### Drop the tables before closing out the sessions

In [150]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS users")
session.execute("DROP TABLE IF EXISTS events")
session.execute("DROP TABLE IF EXISTS artist")

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()